In [1]:
versioninfo()

Julia Version 0.5.0-rc0+0
Commit 633443c (2016-08-02 00:53 UTC)
Platform Info:
  System: Linux (x86_64-redhat-linux)
  CPU: Intel(R) Core(TM) i5-3470 CPU @ 3.20GHz
  WORD_SIZE: 64
  BLAS: libopenblas (USE64BITINT DYNAMIC_ARCH NO_AFFINITY Sandybridge)
  LAPACK: libopenblas64_
  LIBM: libopenlibm
  LLVM: libLLVM-3.7.1 (ORCJIT, ivybridge)


In [2]:
import Base: significand_mask, exponent_one
f1(u::UInt64) = reinterpret(Float64, exponent_one(Float64) | significand_mask(Float64) & u) - 1.0
# use the least 52 bits to generate uniform distribution in [1, 2). And then substracted by 1.0.

f1 (generic function with 1 method)

In [3]:
import Base: llvmcall
@inline cttz(x::UInt64) = llvmcall(
    ("""declare i64 @llvm.cttz.i64(i64, i1)""",
     """%2 = call i64 @llvm.cttz.i64(i64 %0, i1 false)
     ret i64 %2"""), Int64, Tuple{UInt64}, x)


function f2(u::UInt64)
    u &= 0x001f_ffff_ffff_ffff
    p = cttz(u)
    if p == 64
        return 0.0
    end
    m = (u >> (p + 1)) << p
    ldexp(0x1p52 + m, -53 - p)
end

f2 (generic function with 1 method)

See https://github.com/art4711/random-double/issues/2

In [4]:
function test_speed(f, a)
    n = length(a)
    s = 0.0
    for i = 1:n
        s += f(a[i])
    end
    s
end
test_speed1(a) = test_speed(f1, a)
test_speed2(a) = test_speed(f2, a)

test_speed2 (generic function with 1 method)

In [5]:
a = rand(UInt64, 10_000_000);

In [6]:
test_speed1(a)

4.999896374054646e6

In [7]:
test_speed2(a)

5.000988653906729e6

In [8]:
f3(x::UInt64) = x * 0x1p-64
test_speed3(a) = test_speed(f3, a)
test_speed3(a)

4.99880648324499e6

In [9]:
f4(x::UInt64) = (x >> 11) * 0x1p-53
test_speed4(a) = test_speed(f4, a)
test_speed4(a)

4.99880648324499e6

In [10]:
f5(x::UInt64) = (x & 0x001f_ffff_ffff_ffff) * 0x1p-53
test_speed5(a) = test_speed(f5, a)
test_speed5(a)

5.001171687027291e6

In [11]:
@time test_speed1(a)
@time test_speed2(a)
@time test_speed3(a)
@time test_speed4(a)
@time test_speed5(a)

  0.010534 seconds (134 allocations: 7.719 KB)
  0.108693 seconds (5 allocations: 176 bytes)
  0.014659 seconds (5 allocations: 176 bytes)
  0.032913 seconds (5 allocations: 176 bytes)
  0.031085 seconds (5 allocations: 176 bytes)


5.001171687027291e6

In [12]:
foo1(x) = reinterpret(Float64, cttz(x))
bar1(a) = test_speed(foo1, a)
bar1(a)
@time bar1(a)

  0.016655 seconds (5 allocations: 176 bytes)


4.9397543e-317

The `cttz` function just takes more time than f1 (even more than f3)...

The C program is also slow:
```zsh
Glaceon ➜  build git:(master) ✗ gcc r0to1.c -lbsd -O3 -DN=100000000 -o r0to1.out && ./r0to1.out
4039.254000 ms
792.179000 ms
50001405.769910

Glaceon ➜  build git:(master) ✗ clang r0to1.c -lbsd -O3 -DN=100000000 -o r0to1.out && ./r0to1.out
3886.342000 ms
929.665000 ms
49998428.474505
```
The first elapsed time value denotes to the time to generate random numbers by `arc4random_buf`. The second is the time to do `r0to1(a)`. It's just a little faster than the Julia functions.
BTW, the time performance of PCG is:
```julia
julia> using RNG.PCG

julia> r = PCGStateSetseq(UInt128, RNG.PCG.PCG_XSH_RS);

julia> a = Array{UInt64}(100_000_000);

julia> @time rand!(r, a);
  0.383423 seconds (4 allocations: 160 bytes)

julia> foo(r) = rand(r, UInt64) & ((1 << 53) - 1)
foo (generic function with 1 method)

julia> bar(r, a) = for i = 1:length(a)
           @inbounds a[i] = foo(r)
       end
bar (generic function with 1 method)

julia> @time bar(r, a);
  0.464137 seconds (4 allocations: 160 bytes)
```